<a href="https://colab.research.google.com/github/COMM599-Fall2023/fall_2023/blob/main/student%20folders/fungn/final/fungn_book_recommender_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# open romance interactions file
romance_interactions = pd.read_csv('/content/drive/MyDrive/COMM599_fungn_final_datasets/romance_interactions_chunk.csv')

In [4]:
# Remove unnecessary columns

romance_interactions = romance_interactions.drop(columns=['Unnamed: 0.1', 'Unnamed: 0', 'review_id', 'is_read', 'review_text_incomplete', 'date_added', 'date_updated', 'read_at', 'started_at'])

In [5]:
# open romance books file
path = "/content/drive/MyDrive/COMM599_fungn_final_datasets/romance_final_df.csv"
romance_books = pd.read_csv(path)
romance_books.head()

,index,isbn,text_reviews_count,series,country_code,language_code,popular_shelves,asin,is_ebook,average_rating,...,publication_month,edition_information,publication_year,url,image_url,book_id,ratings_count,work_id,title,title_without_series
0,1,NaN,21,['811663'],US,en-US,"[{'count': '598', 'name': 'to-read'}, {'count'...",B01BLJGA9S,True,4.23,...,NaN,NaN,NaN,https://www.goodreads.com/book/show/29074693-p...,https://s.gr-assets.com/assets/nophoto/book/11...,29074693,149,46079519,"Prowled Darkness (Dante's Circle, #7)","Prowled Darkness (Dante's Circle, #7)"
1,2,1597371289,8,[],US,eng,"[{'count': '16215', 'name': 'classics'}, {'cou...",NaN,False,3.99,...,9.0,NaN,2005.0,https://www.goodreads.com/book/show/3209316-emma,https://s.gr-assets.com/assets/nophoto/book/11...,3209316,42,3360164,Emma,Emma
2,3,NaN,27,['938303'],US,en-GB,"[{'count': '25', 'name': 'to-read'}, {'count':...",B01HX6PENG,True,4.31,...,NaN,NaN,NaN,https://www.goodreads.com/book/show/30838933-g...,https://s.gr-assets.com/assets/nophoto/book/11...,30838933,139,51437308,"Guardian Cougar (Finding Fatherhood, #2)","Guardian Cougar (Finding Fatherhood, #2)"
3,4,NaN,15,[],US,eng,"[{'count': '1492', 'name': 'to-read'}, {'count...",B013Q70BQG,True,3.98,...,NaN,NaN,NaN,https://www.goodreads.com/book/show/27419760-w...,https://images.gr-assets.com/books/1457306424m...,27419760,167,46003673,Wedding Girl,Wedding Girl
4,5,NaN,1,['584334'],US,en-US,"[{'count': '341', 'name': 'to-read'}, {'count'...",B00HWK35OY,True,3.79,...,NaN,NaN,NaN,https://www.goodreads.com/book/show/20566082-a...,https://s.gr-assets.com/assets/nophoto/book/11...,20566082,53,32222419,"A Kitty in the Lion's Den (Sweet Water, #3)","A Kitty in the Lion's Den (Sweet Water, #3)"


In [6]:
# remove unnecessary columns

romance_books = romance_books.drop(columns= ['index', 'asin', 'kindle_asin', 'isbn', 'isbn13', 'text_reviews_count', 'series', 'country_code', 'language_code', 'popular_shelves', 'is_ebook', 'publication_month', 'edition_information', 'url', 'image_url', 'work_id', 'link', 'publication_day', 'description', 'format'])
romance_books.head()

,average_rating,similar_books,authors,publisher,num_pages,publication_year,book_id,ratings_count,title,title_without_series
0,4.23,"['25515353', '20483269', '25650829', '18913492...","[{'author_id': '5360266', 'role': ''}]",NaN,NaN,NaN,29074693,149,"Prowled Darkness (Dante's Circle, #7)","Prowled Darkness (Dante's Circle, #7)"
1,3.99,"['31242', '374380', '20564', '383206', '7891',...","[{'author_id': '1265', 'role': ''}]",Brilliance Audio,544.0,2005.0,3209316,42,Emma,Emma
2,4.31,[],"[{'author_id': '90411', 'role': ''}, {'author_...",NaN,NaN,NaN,30838933,139,"Guardian Cougar (Finding Fatherhood, #2)","Guardian Cougar (Finding Fatherhood, #2)"
3,3.98,"['386097', '7774253', '576667', '471907', '431...","[{'author_id': '47231', 'role': ''}]",NaN,NaN,NaN,27419760,167,Wedding Girl,Wedding Girl
4,3.79,"['22044544', '23694100', '18243587', '13542030...","[{'author_id': '3330668', 'role': ''}]",NaN,NaN,NaN,20566082,53,"A Kitty in the Lion's Den (Sweet Water, #3)","A Kitty in the Lion's Den (Sweet Water, #3)"


In [7]:
# Merging both dataframes by book_id

df = pd.merge(romance_books, romance_interactions, on='book_id', how='inner')
df.head()

,average_rating,similar_books,authors,publisher,num_pages,publication_year,book_id,ratings_count,title,title_without_series,user_id,rating
0,3.89,"['1224327', '2123409', '691005', '959223', '10...","[{'author_id': '12935', 'role': ''}]",Zebra,507.0,1994.0,958988,352,"Princess of Fire (Fire, #1)","Princess of Fire (Fire, #1)",3f748d5642fbcc28e93dda06b74e87e2,5
1,3.89,"['1224327', '2123409', '691005', '959223', '10...","[{'author_id': '12935', 'role': ''}]",Zebra,507.0,1994.0,958988,352,"Princess of Fire (Fire, #1)","Princess of Fire (Fire, #1)",4a4a8e60702e892d9b35c47860e97e82,4
2,4.26,"['351572', '362206', '362204', '357096', '2210...","[{'author_id': '2614', 'role': ''}]",Ellora's Cave,358.0,2007.0,388678,1347,"Shameless Embraces (Bound Hearts, #6-7)","Shameless Embraces (Bound Hearts, #6-7)",a45e858447e808b39fe3b56f381440d7,5
3,4.36,"['29992024', '19459671', '29242560', '25865290...","[{'author_id': '7246093', 'role': ''}]",NaN,NaN,2017.0,25501128,1759,"Kept from You (Tear Asunder, #4)","Kept from You (Tear Asunder, #4)",f029b0ae8e28982b9d9dd97181b91ff7,4
4,3.38,"['6336101', '10953518', '11076946', '9993763',...","[{'author_id': '2654606', 'role': ''}]",Burlesque Press,57.0,2011.0,12458821,755,Awakening,Awakening,35dc073dd0025da5f6e8f456925dfab0,3


In [8]:
# Find number of ratings per book

agg_ratings = df.groupby('book_id').agg(number_of_ratings = ('rating', 'count')).reset_index()

In [9]:
# Keep the movies with over 10 ratings
agg_ratings_GT10 = agg_ratings[agg_ratings['number_of_ratings']>10]

In [10]:
# Adding the average_rating column

avg_rating = df['average_rating']
agg_ratings_GT10 = agg_ratings_GT10.join(avg_rating, how='inner')
agg_ratings_GT10.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105 entries, 1 to 11830
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   book_id            105 non-null    int64  
 1   number_of_ratings  105 non-null    int64  
 2   average_rating     105 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 3.3 KB


In [11]:
# Merge the books with greater than 10 reviews with the user ratings

df_GT10 = pd.merge(df, agg_ratings_GT10[['book_id']], on='book_id', how='inner')
df_GT10.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2420 entries, 0 to 2419
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   average_rating        2420 non-null   float64
 1   similar_books         2420 non-null   object 
 2   authors               2420 non-null   object 
 3   publisher             2113 non-null   object 
 4   num_pages             2395 non-null   float64
 5   publication_year      2159 non-null   float64
 6   book_id               2420 non-null   int64  
 7   ratings_count         2420 non-null   int64  
 8   title                 2420 non-null   object 
 9   title_without_series  2420 non-null   object 
 10  user_id               2420 non-null   object 
 11  rating                2420 non-null   int64  
dtypes: float64(3), int64(3), object(6)
memory usage: 245.8+ KB


In [12]:
# print number of unique users

print(df_GT10['user_id'].nunique())

435


In [13]:
# Create a user book matrix
matrix = df_GT10.pivot_table(index='book_id', columns='user_id', values='rating').fillna(0)
matrix.head()

user_id,003e69e29bdb5a25c116c649ac20b6b4,015988b10cec9949eb29663fcac897f5,01d7253e4a3be6c2766d3355d103d862,03204afbcbce0ed6043e854965c6cddb,03302f7aa312ef92b9428745cc6fbbb4,03e38f9fc0708657782c7e5495320b3c,042136e8090a337cb39b7ad1a2c1a7fe,0471745621a43e811e79618b4044b599,04af21ced630a3b2707e7afbfd68fcb4,0604075ea381188c40b5959c28999a93,...,fb3449e43fa0ab7deb90c21e4bf8f1db,fb5f600a386d6ad0e2e296abbba5b3f1,fb6858f48db8ade9d05ad5d8dbf80bad,fcad25c3962b38e8aed4926153420759,fcd4dfb9b2df732c280d954d45bc2491,fd35c5fc9b0a778015d5b3b8f4b58aa4,fd9271d7dc8753acb8c2c5637956635b,fdff24c6d354d0771fe4f38481cbd151,fe4f6676373c52f8bd25b10dca1d8983,fe6876cf4b9327666f859d168ba5d5e8
book_id,,,,,,,,,,,,,,,,,,,,,
1431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,4.0,5.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
2156,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0
3412,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3473,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.0,0.0


In [14]:
# find nearest neighbors

knn_model = NearestNeighbors(metric='cosine', algorithm='auto')
knn_model.fit(matrix)

NearestNeighbors(metric='cosine')

In [15]:
# create index lookup

index_lkp = dict(zip(range(len(matrix)), matrix.index))

In [16]:
# drop duplicate books

title_mapping = df_GT10[['book_id', 'title']].drop_duplicates()

In [17]:
# create title lookup

title_lkp = dict(zip(df_GT10['book_id'].values, df_GT10['title'].values))

In [18]:
def get_similar_books(book, data, index_lkp, model, title_lkp = None, n = 5):
  ## input to this function is the book and number of top similar books you want.
  if type(book) == str:
    book = [key for key, value in title_lkp.items() if value == book][0]
  input = data[data.index == book].values.reshape(1,-1)
  distances, indices = model.kneighbors(input, n_neighbors=n+1)

  results = [index_lkp[indices[0][i]] for i in range(1,len(distances[0]))]
  if title_lkp is not None:
    results = [title_lkp[x] for x in results]
  return(results)

In [19]:
# getting similar books to 'Emma'

get_similar_books("Emma", matrix, index_lkp, knn_model, title_lkp)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


['Mansfield Park',
 'Persuasion',
 'Northanger Abbey',
 'Pride and Prejudice',
 'Wuthering Heights']